In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import sympy as sp

from matplotlib.transforms import Affine2D
from matplotlib.lines import Line2D
from scipy.integrate import solve_ivp

In [ ]:
n = 3 # number of masses
n_springs = n + 1 # number of springs

masses = sp.symbols(f'm1:{n+1}')
ks = sp.symbols(f'k1:{n_springs+1}')

t = sp.symbols('t', real=True)
m, K, k = sp.symbols('m K k', positive=True)

x1, x2 = sp.symbols('x1 x2', cls=sp.Function)

A = sp.zeros(n, n)

for i in range(n):
        if i > 0:
            A[i, i-1] += ks[i]/masses[i]
        
        A[i, i] -= (ks[i]+ks[i+1])/masses[i]

        if i < n-1:
             A[i, i+1] += ks[i+1]/masses[i]

m_values = [(mi, m) for mi in masses]
k_values = [(ki, k) for ki in ks]
k_values[0] = (ks[0], K)
k_values[-1] = (ks[-1], K)

x_eigenvects = []
x_eigenvals = []

for ev, mult, vecs in A.subs(m_values).subs(k_values).eigenvects():
      x_eigenvects.extend(vecs)
      x_eigenvals.extend([ev] * mult)

b = sp.symbols(f'b1:{n+1}')

modes = [b[k] * x_eigenvects[k] * sp.cos(sp.sqrt(-x_eigenvals[k])*t) for k in range(n)]

sol = modes[0]

for mode in modes[1:]:
      sol += mode

initial = sp.zeros(n, 1)
initial[1] = 1
initial[2] = 1

sol_coef = sp.solve(sp.Eq(sol.subs(t, 0), initial))
if type(sol_coef) == list:
      sol_coef = sol_coef[0]

sol_concrete = sol.subs(sol_coef)

paras = {k: 3, K: 10, m: 0.25}
tmax = 8

sol_list = [sol_concrete.subs(paras)[i] for i in range(n)]

sp.plot(*sol_list, (t, 0, tmax))

In [ ]:
A = sp.Matrix([[1], [1]])
A

In [ ]:

m1 = 1
m2 = 1
K = 100
k = 10

x0 = np.array([0.2, 0, 0.05, 0])
tmin = 0
tmax = 10
N = 500

def mass_spring_ODE(t, x):
    x1 = x[0]
    x1dot = x[1]
    x2 = x[2]
    x2dot = x[3]

    return (
        x1dot,
        -(K+k)/m1 * x1 + k/m1 * x2,
        x2dot,
        k/m2 * x1 - (K+k)/m2 * x2
    )

sol = solve_ivp(mass_spring_ODE, [tmin, tmax], x0, t_eval=np.linspace(tmin, tmax,N))

x1 = sol.y[0]
x2 = sol.y[2]
t = sol.t

fig, axs = plt.subplots(2, figsize=(8, 8), gridspec_kw={'height_ratios': [1, 1]})
ax1, ax2 = axs

ax1.plot(t, x1, label='$x_1(t)$')
ax1.legend()
ax1.grid()
ax2.plot(t, x2, label='$x_2(t)$')
ax2.legend()
ax2.grid()
plt.xlabel('time (s)')


In [ ]:
width = 4
height = 0.3

n_masses = 2
N_spring = 20
w_spring = 0.05

m_width = 0.5
m_height = 0.2

fig = plt.figure()
ax = fig.add_subplot(aspect='equal')
ax.set_xlim(0, width)
ax.set_ylim(0, height)

def generate_spring(length, width, n):
    data = np.zeros((2, n+2))
    data[:, -1] = [length, 0]
    for i in range(1, n+1):
        data[0, i] = length * (2*i-1)/(2*n)
        data[1, i] = -width if i % 2 else width
    return data

l0 = (width - n_masses*m_width)/(n_masses + 1)

# mass positions
p1 = l0
p2 = width - l0 - m_width

ps = [0, l0 + m_width, 2 * (l0 + m_width)]

rect1 = ax.add_patch(plt.Rectangle((p1, 0), m_width, m_height))
rect2 = ax.add_patch(plt.Rectangle((p2, 0), m_width, m_height))

data = np.append(generate_spring(l0, w_spring, N_spring), np.ones((1, N_spring+2)), axis=0)

springs = []

for p in ps:
    A = Affine2D().translate(p, m_height/2).get_matrix()
    data_plot = A @ data
    spring = Line2D(data_plot[0, :], data_plot[1, :], color='r')
    ax.add_line(spring)
    springs.append(spring)

def animate(i):

    p1 = l0+x1[i]
    p2 = width-l0-m_width+x2[i]

    ps = [0, l0+m_width+x1[i], 2*(l0+m_width)+x2[i]]
    ls = [l0+x1[i], l0+x2[i]-x1[i], l0-x2[i]]

    rect1.set_x(p1)
    rect2.set_x(p2)

    for i, (p, l) in enumerate(zip(ps, ls)):
        data = np.append(generate_spring(l, w_spring, N_spring), np.ones((1, N_spring+2)), axis=0)
        A = Affine2D().translate(p, m_height/2).get_matrix()
        data_plot = A @ data
        springs[i].set_data(data_plot[0, :], data_plot[1, :])

ani = animation.FuncAnimation(fig, animate, frames=len(t))
ffmpeg_writer = animation.FFMpegWriter(fps=20)
ani.save('coupled_masses.gif', writer=ffmpeg_writer)